In [7]:
from datasets import load_dataset
from transformers import AutoModelForMaskedLM, AutoModelForSequenceClassification, RobertaForSequenceClassification, AutoTokenizer
from tokenizers import Tokenizer
model = AutoModelForMaskedLM.from_pretrained("ufal/robeczech-base")
model2 = AutoModelForSequenceClassification.from_pretrained("ufal/robeczech-base")
tok = AutoTokenizer.from_pretrained("ufal/robeczech-base")

Some weights of the model checkpoint at ufal/robeczech-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ufal/robeczech-base and are newly initialized: 

In [63]:
tok = dm.tokenizer

In [66]:
tok.vocab_files_names

{'vocab_file': 'vocab.json',
 'merges_file': 'merges.txt',
 'tokenizer_file': 'tokenizer.json'}

In [5]:
model2

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(51961, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [7]:
model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(51961, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [ ]:
lm_head.decoder = None

In [25]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:


from datamodules import NewsDataModuleForLM
dm = NewsDataModuleForLM(cache_dir="/home/kydliceh/.cache", batch_size=2, limit=1000, tokenizer="ufal/robeczech-base")

In [2]:
dm.prepare_data()

In [11]:
from datasets import load_from_disk

In [23]:
dst = load_from_disk("/home/kydliceh/.cache/LM/512/train")

In [30]:
dst[1:4]["input_ids"].shape

torch.Size([3, 512])

In [34]:
import torch
maximum = 0
minimum = 1000
for x in dst["input_ids"]:
    maximum = max(maximum, x.shape[0])
    minimum = min(minimum, x.shape[0])


In [39]:
torch.max(dst[140:150]["input_ids"])

tensor(51926)

In [42]:
from datamodules import NewsDataModuleForLM
dm = NewsDataModuleForLM(cache_dir="/home/kydliceh/.cache", batch_size=8, tokenizer="ufal/robeczech-base")

In [48]:
len(

tensor(51996)

In [47]:
maximum = 0
for x in dm.train_dataloader():
    maximum = max(maximum, torch.max(x["input_ids"]))
    assert torch.all(x["attention_mask"] == 1)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│   1 maximum = 0                                                                                  │
│ ❱ 2 for x in dm.train_dataloader():                                                              │
│   3 │   maximum = max(maximum, torch.max(x["input_ids"]))                                        │
│   4                                                                                              │
│                                                                                                  │
│ /lnet/aic/personal/kydliceh/non_runable/NLP_venv/lib/python3.10/site-packages/torch/utils/data/d │
│ ataloader.py:634 in __next__                                                                     │
│                                                                                                  │
│    631 │   │   │   if self._sampler_iter is None:                                                │
│    632 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    633 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  634 │   │   │   data = self._next_data()                                                      │
│    635 │   │   │   self._num_yielded += 1                                                        │
│    636 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    637 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /lnet/aic/personal/kydliceh/non_runable/NLP_venv/lib/python3.10/site-packages/torch/utils/data/d │
│ ataloader.py:1329 in _next_data                                                                  │
│                                                                                                  │
│   1326 │   │   │   │   return self._process_data(data)                                           │
│   1327 │   │   │                                                                                 │
│   1328 │   │   │   assert not self._shutdown and self._tasks_outstanding > 0                     │
│ ❱ 1329 │   │   │   idx, data = self._get_data()                                                  │
│   1330 │   │   │   self._tasks_outstanding -= 1                                                  │
│   1331 │   │   │   if self._dataset_kind == _DatasetKind.Iterable:                               │
│   1332 │   │   │   │   # Check for _IterableDatasetStopIteration                                 │
│                                                                                                  │
│ /lnet/aic/personal/kydliceh/non_runable/NLP_venv/lib/python3.10/site-packages/torch/utils/data/d │
│ ataloader.py:1295 in _get_data                                                                   │
│                                                                                                  │
│   1292 │   │   │   # need to call `.task_done()` because we don't use `.join()`.                 │
│   1293 │   │   else:                                                                             │
│   1294 │   │   │   while True:                                                                   │
│ ❱ 1295 │   │   │   │   success, data = self._try_get_data()                                      │
│   1296 │   │   │   │   if success:                                                               │
│   1297 │   │   │   │   │   return data                                                           │
│   1298                                                     